In [1]:
#  ĐỀ BÀI:
# Deadline nộp bài: 12h ngày 11/08/2025
# Title email: MSE K23HN_DAM501_Fullname
# Gửi link drive ở chế độ shared tới email: hungkbkhn@gmail.com

# Nội dung trong Drive:
# files dataset
# file notebook
# ảnh chụp màn hình minh chứng hoàn thành Khóa học Coursera

# Trình bày (online):
# vào buổi cuối 16/08 theo lịch

# Yêu cầu thực hiện:
# Mỗi bạn chọn 1 dataset tại: https://www.kaggle.com/discussions/general/210203
# (Tự đưa ra giá trị min support, min confidence phù hợp)

# Nhiệm vụ lập trình (bằng Python):
# Khai thác tập phổ biến sử dụng giải thuật maxFP-growth
# → Khai thác luật kết hợp
# → Áp dụng cho ID sinh viên chẵn

# Khai thác tập phổ biến sử dụng giải thuật Apriori
# → Khai thác luật kết hợp
# → Áp dụng cho ID sinh viên lẻ

In [2]:
# Link dataset: https://www.kaggle.com/datasets/patricklford/mass-disasters-in-vietnam-from-1900-to-2024

# Đề tài: Association Rule Mining of Natural Disaster Patterns and Damage Severity in Vietnam (1953–2024)

# 4 phần phải làm
# 1. Làm sạch & phân tích sơ bộ dữ liệu
# 2. Phân loại damage level, gán mùa, phân theo vùng
# 3. Chạy Apriori, sinh luật kết hợp
# 4. Viết báo cáo, slide và hoàn thiện mô tả phân tích

In [3]:
# Path
path = "disaster_in_vietnam.csv"

In [4]:
# Read file
import pandas as pd
df = pd.read_csv(path, encoding="utf-8")

In [5]:
# Chọn các cột cần thiết
df = df[
    [
        'Disaster Group', 'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype',
        'Location', 'Start Year', 'Start Month',
        'Total Deaths', 'No. Injured', 'Total Affected', "Total Damage ('000 US$)"
    ]
]

In [6]:
# Đổi tên cột cho ngắn gọn và dễ xử lý
df.columns = ['group', 'subgroup', 'type', 'subtype', 'location', 'year', 'month',
              'deaths', 'injured', 'affected', 'damage']

In [7]:
# Chuyển cột số thành kiểu numeric (tránh lỗi khi có chuỗi)
df['deaths'] = pd.to_numeric(df['deaths'], errors='coerce')
df['injured'] = pd.to_numeric(df['injured'], errors='coerce')
df['affected'] = pd.to_numeric(df['affected'], errors='coerce')
df['damage'] = pd.to_numeric(df['damage'], errors='coerce')

In [8]:
# Bỏ dòng không có thông tin về loại thiên tai hoặc thời gian
df = df.dropna(subset=['type', 'year', 'month'])

In [9]:
# Chuyển cột 'location' về chữ thường để dễ xử lý
df['location_clean'] = df['location'].astype(str).str.lower()
print(df['location_clean'])

0                                         southern coast
1                                                    nan
2                                   saigon, mekong delta
3                                        china sea coast
4                                                    nan
                             ...                        
330    quang nam, quang tri, phu yen, phu loc distric...
331                                  kien giang province
332                     lai chau and dien bien provinces
333                                                hanoi
334     quang tri, thua thien hue and ha tinh provinces 
Name: location_clean, Length: 333, dtype: object


In [10]:
# Cai dat cac module can thiet de download file, va chuan hoa ten tinh
!pip install unidecode
!pip install rapidfuzz
!pip install openpyxl
!pip install xlsxwriter

In [11]:
# Chuẩn hoá tên tỉnh

import re
import unidecode
from rapidfuzz import fuzz, process
import pandas as pd

province_list = [
    'ha noi', 'ha giang', 'cao bang', 'bac kan', 'tuyen quang', 'lao cai', 
    'dien bien', 'lai chau', 'son la', 'yen bai', 'hoa binh', 'thai nguyen',
    'lang son', 'quang ninh', 'bac giang', 'phu tho', 'vinh phuc', 'bac ninh',
    'hai duong', 'hai phong', 'hung yen', 'thai binh', 'ha nam',
    'nam dinh', 'ninh binh', 'thanh hoa', 'nghe an', 'ha tinh', 'quang binh',
    'quang tri', 'thua thien hue', 'da nang', 'quang nam', 'quang ngai',
    'binh dinh', 'phu yen', 'khanh hoa', 'ninh thuan', 'binh thuan',
    'kon tum', 'gia lai', 'dak lak', 'dak nong', 'lam dong', 'ho chi minh',
    'binh phuoc', 'tay ninh', 'binh duong', 'dong nai', 'ba ria vung tau',
    'long an', 'tien giang', 'ben tre', 'tra vinh', 'vinh long', 'dong thap',
    'an giang', 'kien giang', 'can tho', 'hau giang', 'soc trang', 'bac lieu',
    'ca mau'
]

# Làm sạch văn bản
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = unidecode.unidecode(text.lower())
    text = re.sub(r'[^a-z\s]', ' ', text)       # loại bỏ ký tự đặc biệt
    text = re.sub(r'\s+', ' ', text).strip()    # chuẩn hóa khoảng trắng
    return text

# Chia câu thành các cụm từ n-gram để tìm tỉnh
def extract_all_provinces(text):
    cleaned = clean_text(text)
    words = cleaned.split()
    found = set()

    for n in range(1, 4):  # kiểm tra các cụm từ 1, 2, 3 từ
        for i in range(len(words) - n + 1):
            phrase = ' '.join(words[i:i+n])
            match, score, _ = process.extractOne(phrase, province_list, scorer=fuzz.ratio)
            if score >= 90:
                found.add(match)

    return ', '.join(sorted(found)) if found else None

df['location_replace'] = df['location_clean'].apply(extract_all_provinces)
print(df)


             group                subgroup                  type  \
0          Natural          Meteorological                 Storm   
1          Natural          Meteorological                 Storm   
2          Natural              Biological              Epidemic   
3          Natural          Meteorological                 Storm   
4          Natural            Hydrological                 Flood   
..             ...                     ...                   ...   
330        Natural            Hydrological                 Flood   
331        Natural          Meteorological                 Storm   
332        Natural            Hydrological                 Flood   
333  Technological  Miscellaneous accident  Fire (Miscellaneous)   
334        Natural          Meteorological                 Storm   

                  subtype                                           location  \
0        Tropical cyclone                                     Southern coast   
1        Tropical cyclo

In [12]:
# Xuất ra file Excel
df[['location_replace']].to_excel('dia_phuong_khac.xlsx', index=False)
print("Đã xuất file dia_phuong_khac.xlsx thành công!")

Đã xuất file dia_phuong_khac.xlsx thành công!


In [13]:
# Dictionary nhữung item không trong danh sách
location_mapping_simple = {
    'southern coast': 'long an, tien giang, dong thap, vinh long, tra vinh, can tho, hau giang, soc trang, ben tre, an giang, kien giang, bac lieu, ca mau',
    'saigon': 'ho chi minh',
    'mekong delta': 'ho chi minh',
    'china sea coast': 'da nang, quang nam, quang ngai, binh dinh, phu yen, khanh hoa, ninh thuan, binh thuan',
    'near cambodia border': 'tay ninh, an giang, kien giang',
    'north': 'ha noi, ha giang, cao bang, bac kan, tuyen quang, lao cai, dien bien, lai chau, son la, yen bai, hoa binh, thai nguyen, lang son, quang ninh, bac giang, phu tho, vinh phuc, bac ninh, hai duong, hai phong, hung yen, thai binh, ha nam, nam dinh, ninh binh, thanh hoa',
    'near saigon': 'binh duong, dong nai, long an',
    'central': 'nghe an, ha tinh, quang binh, quang tri, thua thien hue, da nang, quang nam, quang ngai, binh dinh, phu yen, khanh hoa, ninh thuan, binh thuan, kon tum, gia lai, dak lak, dak nong, lam dong',
    'south': 'ho chi minh, binh phuoc, tay ninh, binh duong, dong nai, ba ria vung tau, long an, tien giang, ben tre, tra vinh, vinh long, dong thap, an giang, kien giang, can tho, hau giang, soc trang, bac lieu, ca mau',
    'central - north regions': 'thanh hoa, nghe an, ha tinh, quang binh, quang tri, thua thien hue',
    'hue': 'thua thien hue',
    'nha trang': 'khanh hoa',
    'halong bay': 'quang ninh',
    'chuong my': 'ha noi',
    'near hoi an': 'quang nam',
    'near con dao island': 'ba ria vung tau',
    'binh tri thien': 'thua thien hue, quang tri, quang binh',
    'nghe tinh': 'nghe an, ha tinh',
    'nghia binh': 'binh dinh, quang ngai',
    'phu khahu': 'khanh hoa',
    'cu long coast': 'long an, tien giang, dong thap, vinh long, tra vinh, can tho, hau giang, soc trang, ben tre, an giang, kien giang, bac lieu, ca mau',
    'tonkin gulf': 'quang ninh, hai phong',
    'thi vai river': 'ba ria vung tau',
    'thinh dan village': 'thai nguyen',
    'kien kang': 'kien giang',
    'lang khe': 'nghe an',
    'near mui ne': 'binh thuan',
    'near ba ria-vung tau': 'ba ria vung tau',
    'tan dan': 'hai duong',
    'krong pak': 'dak lak',
    'hanoã¯': 'ha noi',
    'muong kuon': 'lao cai',
    'bao xat': 'lao cai',
    'bao thang': 'lao cai',
    'sa pa': 'lao cai',
    'thuan an': 'thua thien hue',
    'chau binh': 'nghe an',
}

In [16]:
# Danh sách các tỉnh theo vùng
north_provinces = [
    'ha noi', 'ha giang', 'cao bang', 'bac kan', 'tuyen quang', 'lao cai',
    'dien bien', 'lai chau', 'son la', 'yen bai', 'hoa binh', 'thai nguyen',
    'lang son', 'quang ninh', 'bac giang', 'phu tho', 'vinh phuc', 'bac ninh',
    'hai duong', 'hai phong', 'hung yen', 'thai binh', 'ha nam', 'nam dinh',
    'ninh binh'
]

central_provinces = [
    'thanh hoa', 'nghe an', 'ha tinh', 'quang binh', 'quang tri', 'thua thien hue',
    'da nang', 'quang nam', 'quang ngai', 'binh dinh', 'phu yen', 'khanh hoa',
    'ninh thuan', 'binh thuan', 'kon tum', 'gia lai', 'dak lak', 'dak nong', 'lam dong'
]

south_provinces = [
    'ho chi minh', 'binh phuoc', 'tay ninh', 'binh duong', 'dong nai', 'ba ria vung tau',
    'long an', 'tien giang', 'ben tre', 'tra vinh', 'vinh long', 'dong thap',
    'an giang', 'kien giang', 'can tho', 'hau giang', 'soc trang', 'bac lieu', 'ca mau'
]

# Sinh tự động dictionary ánh xạ
province_to_region = {}
province_to_region.update({p: 'Bắc' for p in north_provinces})
province_to_region.update({p: 'Trung' for p in central_provinces})
province_to_region.update({p: 'Nam' for p in south_provinces})

In [17]:
def get_region_from_provinces(province_str):
    if pd.isna(province_str):
        return 'Không xác định'
    provinces = [p.strip() for p in province_str.split(',')]
    regions = set()
    for province in provinces:
        region = province_to_region.get(province)
        if region:
            regions.add(region)
    if not regions:
        return 'Không xác định'
    if len(regions) == 1:
        return regions.pop()
    return ', '.join(sorted(regions))

df['regions'] = df['location_replace'].apply(get_region_from_provinces)

In [18]:
# Tạo cột regions từ location_replace

# Filter theo region "Không xác định" và xóa records có location_replace null
df = df[~((df['regions'] == 'Không xác định') & (df['location_replace'].isna()))]

In [29]:
# Xuất ra file Excel
df.to_excel('dia_phuong.xlsx', index=False)
print("Đã xuất file dia_phuong.xlsx thành công!")

Đã xuất file dia_phuong.xlsx thành công!


In [ ]:
# Đến thời điểm này thì đã xử lý xong cột location. Cột data chuẩn là location replace.

In [24]:
# Phân loại damage level
def classify_damage(damage):
    if pd.isna(damage):
        return 'Không xác định'
    if damage < 1000:
        return 'Nhỏ'
    elif damage < 10000:
        return 'Trung bình'
    elif damage < 100000:
        return 'Lớn'
    else:
        return 'Rất lớn'

df['damage_level'] = df['damage'].apply(classify_damage)

C:\Users\HP\AppData\Local\Temp\ipykernel_10748\2640053727.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['damage_level'] = df['damage'].apply(classify_damage)


In [25]:
# Xuất ra file Excel
df['damage_level'].to_excel('dia_phuong.xlsx', index=False)
print("Đã xuất file dia_phuong.xlsx thành công!")

Đã xuất file dia_phuong.xlsx thành công!


In [26]:
# Gán mùa theo tháng
def get_season(month):
    if pd.isna(month):
        return 'Không xác định'
    month = int(month)
    if month in [12, 1, 2]:
        return 'Đông'
    elif month in [3, 4, 5]:
        return 'Xuân'
    elif month in [6, 7, 8]:
        return 'Hạ'
    elif month in [9, 10, 11]:
        return 'Thu'
    else:
        return 'Không xác định'

df['season'] = df['month'].apply(get_season)

C:\Users\HP\AppData\Local\Temp\ipykernel_10748\818358701.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['season'] = df['month'].apply(get_season)


In [28]:
# Xuất ra file Excel
df['season'].to_excel('dia_phuong.xlsx', index=False)
print("Đã xuất file dia_phuong.xlsx thành công!")

Đã xuất file dia_phuong.xlsx thành công!


In [ ]:
# Bắt đầu chạy giải thuật Apriori, sinh luật kết hợp

In [30]:
# Cài đặt thư viện cần thiết
!pip install mlxtend

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   ------- -----------------------

In [31]:
# Chuẩn bị dữ liệu cho Apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Chọn các cột phân loại phù hợp
cols = ['group', 'type', 'damage_level', 'season', 'regions']
df_apriori = df[cols].astype(str)

# Chuyển mỗi dòng thành 1 list các thuộc tính
transactions = df_apriori.values.tolist()

In [32]:
# Mã hoá dữ liệu
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

In [33]:
# Chạy Apriori để tìm tập phổ biến
frequent_itemsets = apriori(df_encoded, min_support=0.05, use_colnames=True)

# Sinh luật kết hợp
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

# Xem top 10 luật mạnh nhất
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values('confidence', ascending=False).head(10))

          antecedents       consequents   support  confidence      lift
2        (Bắc, Trung)         (Natural)  0.149813         1.0  1.208145
97  (Lớn, Flood, Thu)         (Natural)  0.074906         1.0  1.208145
4             (Flood)         (Natural)  0.355805         1.0  1.208145
8              (Road)  (Không xác định)  0.063670         1.0  2.053846
12              (Lớn)         (Natural)  0.205993         1.0  1.208145
16            (Storm)         (Natural)  0.419476         1.0  1.208145
15          (Rất lớn)         (Natural)  0.142322         1.0  1.208145
31       (Bắc, Storm)         (Natural)  0.097378         1.0  1.208145
21             (Road)   (Technological)  0.063670         1.0  5.804348
25       (Flood, Bắc)         (Natural)  0.078652         1.0  1.208145


In [34]:
rules.to_excel('association_rules.xlsx', index=False)
print("Đã xuất luật kết hợp ra association_rules.xlsx")

Đã xuất luật kết hợp ra association_rules.xlsx
